### Task 1. 

#### MapReduce

Use MapReduce to calculate the average housing price in each zip code and filter out records in which gross sqft is less than 500 and sale price is less than 100000. Round the average housing price to 2 decimals, and sort the output by housing price ascending.

In [3]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=2076e0f1453a3a334cd555d2d273a968c2b43acca6628c65ec70e6b1d30908d3
  Stored in directory: c:\users\86746\appdata\local\pip\cache\wheels\a1\b6\7c\0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [1]:
import wget
import pandas as pd
from functools import reduce
import csv
import multiprocessing
from multiprocessing import Pool

In [3]:
!wget -O housingSalseSample.csv https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv

--2022-03-08 16:43:25--  https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CUSP2022ADS/Data/main/housingSalseSample.csv [following]
--2022-03-08 16:43:25--  https://raw.githubusercontent.com/CUSP2022ADS/Data/main/housingSalseSample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6167298 (5.9M) [text/plain]
Saving to: 'housingSalseSample.csv'

     0K .......... .......... .......... .......... ..........  0% 4.41M 1s
    50K .......... .......... .......... .......... ..........  1% 8.02M 1s
   100K .......... .......... .....

In [262]:
wget.download('https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv')

 67% [.................................................                         ] 4145152 / 6167298

100% [..........................................................................] 6167298 / 6167298

In [263]:
# code here

In [20]:
def returnInfo(record): #extract necessary fields from a data record (raw)
    return(record['ZIP CODE'],record['GROSS SQUARE FEET'],record['SALE PRICE'])

def filtergross(record): #filter the records to keep only the year 2019
    # filter function must return True or False
    price = record[1]
    gross =''.join(price.split())
    gross=gross.zfill(1)
    gross=float(gross)
    if gross<=3000:
        return True
    else:
        return False

def filtersale(record): #filter the records to keep only the year 2019
    # filter function must return True or False
    saleprice = record[2]
    sale =''.join(saleprice.split())
    sale=sale.zfill(1)
    sale=float(sale)
    if sale<=100000:
        return True
    else:
        return False

def zipsList(result,record):
    zips, sales = record[0],record[2]
    sales=float(sales)
    result[zips] = result.get(zips,[]) + [sales]
    return result
def averageprice(record):
    zips, sales = record[0], record[1]
    return [zips, round(sum(sales)/len(sales),2)]

In [21]:
with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output1 = list(map(returnInfo,reader))
output1[:3]

[('10024.0', '0.0', '551000'),
 ('11372.0', '0.0', '338000'),
 ('11219.0', '3861.0', '1250000')]

In [22]:
output2 = list(filter(filtergross, output1))
output2[:3]

[('10024.0', '0.0', '551000'),
 ('11372.0', '0.0', '338000'),
 ('11374.0', '0.0', '253000')]

In [23]:
output3 = list(filter(filtersale, output2))
output3[:3]

[('11354.0', '0.0', '0'),
 ('10031.0', '0.0', '63131'),
 ('11369.0', '1140.0', '0')]

In [24]:
output4 = list(map(averageprice,list(reduce(zipsList,output3,{}).items())))
output4[:4]

[['11354.0', 2111.09],
 ['10031.0', 20567.73],
 ['11369.0', 6065.59],
 ['10128.0', 1390.82]]

In [25]:
output = list(map(lambda x: (x[0],x[1]),output4))

In [26]:
output=sorted(output,reverse=True,key=lambda t:t[1])

In [27]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)
    

grading(output)

,zipcode,price
55,10462.0,28056.242188
76,10468.0,23571.739130
113,10001.0,21844.939394
1,10031.0,20567.727273
152,10459.0,19926.260870


### Task 2. 
#### multiprocessing and Chunk

In [1]:
!wget -O ADSSession2Task2Data.csv https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [2]:
wget.download('https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv')

100% [............................................................................] 279369 / 279369

'ADSSession2Task2Data.csv'

Apply the chunk and MapReduce method to sum up each type's value in the synthetic dataset—only keep records which $Select$ value is 1, and read 1000 lines in each chunk. Use multiprocessing package to deploy the task to multi-cores. The output should only have two columns: type and total value.

Hint: filter function is not available in multiprocessing, implement an if-else function in the map or reduce step.

In [35]:
def returntype(record): #extract necessary fields from a data record (raw)
    return([record['Type'],record['Value'],record['Select']])
    
def sumValue(result,record): #add rides from a new record to the dictionary accumilating type-origin-destination-time ridership
    if record==None:
        pass
    else:
        type, value = record[0], record[1]
        result[type]=result.get(type,0)+float(value)
    return result

def filterSelect(record):
    select=record[2]
    if select=='1':
        return (record[0],record[1])
    else:
        pass

In [2]:
import sys
sys.modules['__main__'].__file__ = 'ipython'

In [3]:
from ipynb.fs.full.my_functions1 import returntype
from ipynb.fs.full.my_functions2 import sumValue
from ipynb.fs.full.my_functions3 import filterSelect

In [4]:
with open('ADSSession2Task2Data.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    result = []
    chunksize = 1000
    pool = Pool(8)
    for i, line in enumerate(reader):
        
        if (i % chunksize == 0 and i > 0):
            output = list(reduce(sumValue,
                         pool.map(filterSelect,
                             pool.map(returntype,chunk)
                            ),
                         {}).items())
            result += output
            chunk = []
        chunk.append(line)
    output = list(reduce(sumValue,
                         pool.map(filterSelect,
                             pool.map(returntype,chunk)
                            ),
                         {}).items())
    result += output
    pool.close()

In [5]:
output=list(reduce(sumValue,result,{}).items())
output

[('3', 294.32549925839936),
 ('0', 306.7099300824752),
 ('1', 316.7932728146583),
 ('9', 291.68146927692595),
 ('6', 302.75696574193273),
 ('7', 280.6029945361827),
 ('8', 274.5970828385255),
 ('5', 302.4264220421208),
 ('4', 320.753814015207),
 ('2', 286.0854868555854)]

In [6]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(output)